In [7]:
import sqlite3
import pandas as pd

In [8]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [9]:
conn = sqlite3.connect('parch-and-posey.db')

In [10]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [11]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

Write a query that uses UNION ALL on two instances (and selecting all columns) of the accounts table. Then inspect the results and answer the subsequent quiz.

In [14]:
pd.read_sql_query(sql = '''
SELECT * FROM accounts
UNION ALL
SELECT * FROM accounts
;
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.169156,-73.849374,Sung Shields,321510
2,1021,Apple,www.apple.com,42.290495,-76.084009,Jodee Lupo,321520
3,1031,Berkshire Hathaway,www.berkshirehathaway.com,40.949021,-75.763898,Serafina Banda,321530
4,1041,McKesson,www.mckesson.com,42.217093,-75.284998,Angeles Crusoe,321540
...,...,...,...,...,...,...,...
697,4461,KKR,www.kkr.com,45.545353,-122.655247,Buffy Azure,321970
698,4471,Oneok,www.oneok.com,45.513513,-122.681500,Esta Engelhardt,321960
699,4481,Newmont Mining,www.newmont.com,45.494117,-122.669460,Khadijah Riemann,321970
700,4491,PPL,www.pplweb.com,45.491720,-122.671880,Deanne Hertlein,321960


Solution:

SELECT *
    FROM accounts

UNION ALL

SELECT *
  FROM accounts

Task 2:

Add a WHERE clause to each of the tables that you unioned in the query above, filtering the first table where name equals Walmart and filtering the second table where name equals Disney. Inspect the results then answer the subsequent quiz.

In [16]:
pd.read_sql_query(sql = '''
SELECT * FROM accounts
WHERE name = 'Walmart'
UNION ALL
SELECT * FROM accounts
WHERE name = 'Disney'
;
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
1,1521,Disney,www.disney.com,41.878800,-74.811026,Timika Mistretta,321600


Solution:

SELECT *
    FROM accounts
    WHERE name = 'Walmart'

UNION ALL

SELECT *
  FROM accounts
  WHERE name = 'Disney'

Task 3:

Performing Operations on a Combined Dataset
Perform the union in your first query (under the Appending Data via UNION header) in a common table expression and name it double_accounts. Then do a COUNT the number of times a name appears in the double_accounts table. If you do this correctly, your query results should have a count of 2 for each name.

In [19]:
pd.read_sql_query(sql = '''
WITH double_accounts AS (SELECT * FROM accounts
UNION ALL
SELECT * FROM accounts)

SELECT name, COUNT(*) count
FROM double_accounts
GROUP BY 1
ORDER BY 2 DESC
;
''', con=conn)

,name,count
0,eBay,2
1,Yum Brands,2
2,Xerox,2
3,Xcel Energy,2
4,World Fuel Services,2
...,...,...
346,AIG,2
347,AES,2
348,AECOM,2
349,ADP,2


Solution:

WITH double_accounts AS (
    SELECT *
      FROM accounts

    UNION ALL

    SELECT *
      FROM accounts
)

SELECT name,
       COUNT(*) AS name_count
 FROM double_accounts 
GROUP BY 1
ORDER BY 2 DESC